In [199]:
%run /Desktop/Share/CUDA_DEVICE_setup.py -n 1

import sys
import torch
from A_speech_command_dataset import SpeechCommandDataset
from module import QuatSincConv1d
from module import SincConv1d
from module import Quantize, QuaternaryConv1d, QuaternaryLinear
from module import DistrLoss
import soundfile as sf
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch.nn.functional as F
print(torch.__version__)
from torchsummary import summary

using GPU ID:2,

1.6.0+cu92


In [200]:
train_set = SpeechCommandDataset()
val_set = SpeechCommandDataset(is_training=False)

print(train_set.__len__())
print(val_set.__len__())

65435
7253


In [201]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [202]:
num_classes = 35

# model = nn.Sequential(
#             SincConv(2, 25, 16000),
#             nn.Conv1d(2, 16, kernel_size=10), 
#             nn.ReLU(),
#             nn.MaxPool1d(kernel_size=10),
#             nn.Conv1d(16, 48, kernel_size=10),
#             nn.ReLU(),
#             nn.MaxPool1d(kernel_size=10),
#             nn.Conv1d(48, 120, kernel_size=8),
#             nn.ReLU(),
#             nn.MaxPool1d(kernel_size=8),
#             nn.Dropout(0.5),
#             nn.Flatten(),
#             nn.Linear(2280,64), 
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(64, num_classes),
#             ).to(device)
# print(model)

In [203]:
class LogAbs(nn.Module):
    def __init__(self):
        super(LogAbs, self).__init__()

    def forward(self, input):
        return torch.log10(torch.abs(input) + 1)


class _Layer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, max_pool_size, use_sinc, cal_distrloss=True):
        super(_Layer, self).__init__()

        self.use_sinc = use_sinc
        self.cal_distrloss = cal_distrloss

        if use_sinc:
            
            self.conv = QuatSincConv1d(
                out_channels=out_channels, kernel_size=kernel_size)
            # self.logabs = LogAbs()
            self.bn = nn.BatchNorm1d(out_channels)
            # self.htanh = nn.Hardtanh(min_val=0.0, max_val=(2**8-1)/(2**8))
            # self.quan = Quantize(8)
            self.htanh = nn.Hardtanh(min_val=0.0)
            self.quan = Quantize(8)
        else:
            
            #self.conv = nn.Conv1d(in_channels=in_channels, out_channels=out_channels,
            #                             kernel_size=kernel_size, stride=stride)
            self.conv = QuaternaryConv1d(in_channels=in_channels, out_channels=out_channels,
                                          kernel_size=kernel_size, stride=stride, num_of_bits=8)
            # self.conv = TernaryConv1d(in_channels=in_channels, out_channels=out_channels,
            #                           kernel_size=kernel_size, stride=stride)
            #self.conv = BinaryConv1d(in_channels=in_channels, out_channels=out_channels,
            #                         kernel_size=kernel_size, stride=stride)
            self.bn = nn.BatchNorm1d(out_channels)
            # self.htanh = nn.Hardtanh(min_val=0.0, max_val=(2**4-1)/(2**4))
            # self.quan = Quantize(4)
            self.htanh = nn.Hardtanh(min_val=0.0)
            self.quan = Quantize(8)

        self.pool = nn.MaxPool1d(max_pool_size)

        if self.cal_distrloss:
            self.distrloss = DistrLoss(out_channels)

    def forward(self, input):
        if self.use_sinc:
            out = self.conv(input)
            # out = self.logabs(out)
            out = self.bn(out)

        else:
            out = self.conv(input)
            out = self.bn(out)

        if self.cal_distrloss:
            dloss = self.distrloss(out)

        out = self.htanh(out)
        out = self.quan(out)

        out = self.pool(out)

        if self.cal_distrloss:
            return out, dloss
        else:
            return out


class QuatenarySincNet(nn.Module):
    def __init__(self, expansion=1.0, cal_distrloss=True):
        super(QuatenarySincNet, self).__init__()

        self.cal_distrloss = cal_distrloss

        self.sincconv = _Layer(
            in_channels=1, out_channels=int(8), kernel_size=50, stride=1, max_pool_size=2, use_sinc=True, cal_distrloss=cal_distrloss)

        self.features = nn.ModuleList()

        self.features.append(_Layer(in_channels=8, out_channels=int(16*expansion),
                                    kernel_size=25, stride=2, max_pool_size=10, use_sinc=False, cal_distrloss=cal_distrloss))

        self.features.append(_Layer(in_channels=int(16*expansion), out_channels=int(32*expansion),
                                    kernel_size=9, stride=1, max_pool_size=10, use_sinc=False, cal_distrloss=cal_distrloss))

        self.features.append(_Layer(in_channels=int(32*expansion), out_channels=int(64*expansion),
                                    kernel_size=9, stride=1, max_pool_size=8, use_sinc=False, cal_distrloss=cal_distrloss))

#         self.gap = nn.AdaptiveAvgPool1d(1)
        self.gap = nn.Flatten()
        self.quan_gap = Quantize(num_of_bits=8)
        # self.out_dim = current_size * 60
        self.fc1 = nn.Linear(int(64*3*expansion), 64, bias=True)
        # self.fc = QuaternaryLinear(int(64*expansion), 10, bias=True)
        self.quan_fc1_w = Quantize(num_of_bits=8)
        
        self.fc2 = nn.Linear(int(64*expansion), 35, bias=True)
        self.quan_fc2_w = Quantize(num_of_bits=8)
        
        self.outs = {}

    def forward(self, input):
        dlosses = []

        if self.cal_distrloss:
            out, dloss = self.sincconv(input)
            dlosses.append(dloss)
            self.outs['sincconv'] = out[0].detach().cpu().numpy()
        else:
            out = self.sincconv(input)

        for i, l in enumerate(self.features):
            if self.cal_distrloss:
                out, dloss = l(out)
                dlosses.append(dloss)
                self.outs['conv'+str(i)] = out[0].detach().cpu().numpy()
            else:
                out = l(out)

        out = self.gap(out)
        out = self.quan_gap(out)
        out = out.view(out.size(0), -1)

        self.outs['gap'] = out[0].detach().cpu().numpy()

        self.fc1.weight.data = F.hardtanh(self.fc1.weight.data)
        self.fc1.weight.data = self.quan_fc1_w(self.fc1.weight.data)
        self.fc1.bias.data = F.hardtanh(self.fc1.bias.data)
        self.fc1.bias.data = self.quan_fc1_w(self.fc1.bias.data)
        out = self.fc1(out)
        #self.outs['fc'] = out[0].detach().cpu().numpy()
        
        self.fc2.weight.data = F.hardtanh(self.fc2.weight.data)
        self.fc2.weight.data = self.quan_fc2_w(self.fc2.weight.data)
        self.fc2.bias.data = F.hardtanh(self.fc2.bias.data)
        self.fc2.bias.data = self.quan_fc2_w(self.fc2.bias.data)
        out = self.fc2(out)
        self.outs['fc2'] = out[0].detach().cpu().numpy()

        if self.cal_distrloss:
            distrloss1 = sum([ele[0] for ele in dlosses]) / len(dlosses)
            distrloss2 = sum([ele[1] for ele in dlosses]) / len(dlosses)

            return out, distrloss1.view(1, 1), distrloss2.view(1, 1)
        else:
            return out

model = QuatenarySincNet().to(device)
model.train()
from torchsummary import summary
# summary(model, (1,16000))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
          Quantize-1                [-1, 1, 51]               0
    QuatSincConv1d-2             [-1, 8, 15950]               0
       BatchNorm1d-3             [-1, 8, 15950]              16
          Hardtanh-4             [-1, 8, 15950]               0
          Quantize-5             [-1, 8, 15950]               0
         MaxPool1d-6              [-1, 8, 7975]               0
            _Layer-7              [-1, 8, 7975]               0
  QuaternaryConv1d-8             [-1, 16, 3976]           3,200
       BatchNorm1d-9             [-1, 16, 3976]              32
         Hardtanh-10             [-1, 16, 3976]               0
         Quantize-11             [-1, 16, 3976]               0
        MaxPool1d-12              [-1, 16, 397]               0
           _Layer-13              [-1, 16, 397]               0
 QuaternaryConv1d-14              [-1, 

In [172]:
def train(model, data_loader, optimizer, epoch, verbose=True):
    model.train()
    loss_avg = 0.0
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output, loss1, loss2 = model(data)
        
        # loss function, you can use other loss function if you want
        loss   = torch.nn.functional.cross_entropy(output, target)
        loss_avg = loss.item()
        
        # do back propagation
        loss.backward()
        #loss1.backward()
        #loss1.backward()
        optimizer.step()
        verbose_step = len(data_loader) // 3
        if batch_idx % verbose_step == 0 and verbose:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader), loss.item()))
    return loss_avg / len(data_loader)

def valid(model, data_loader):
    with torch.no_grad():
        model.eval()
        valid_loss = 0
        correct = 0
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            valid_loss += F.cross_entropy(output[0], target, reduction='sum').item() # sum up batch loss
            pred = output[0].data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).cpu().sum().item() 

        valid_loss /= len(data_loader.dataset)
        print('\nValid set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            valid_loss, correct, len(data_loader.dataset),
            100. * correct / len(data_loader.dataset)))
    return float(correct) / len(data_loader.dataset)

In [173]:
#train
train_data_loader = DataLoader(train_set, batch_size=50, shuffle=True, num_workers=8)
val_data_loader = DataLoader(val_set, batch_size=50, shuffle=True, num_workers=8)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
epochs = 30
acc = 0.0
for epoch in range(epochs):
#     model.train()
#     for batch_idx, (data, target) in enumerate(train_data_loader):
#         data, target = data.to(device), target.to(device)
#         optimizer.zero_grad()
#         output = model(data)
    train(model, train_data_loader, optimizer, epoch)
    accuracy = valid(model, val_data_loader)
    if accuracy > acc:
        acc = accuracy
        print("-------------saving model--------------")
        # save the model
        torch.save(model, "1201_1.pth")

Train Epoch: 0 [0/65435 (0%)]	Loss: 3.591010
Train Epoch: 0 [21800/65435 (33%)]	Loss: 1.363438
Train Epoch: 0 [43600/65435 (67%)]	Loss: 0.969345
Train Epoch: 0 [45780/65435 (100%)]	Loss: 0.631594

Valid set: Average loss: 1.1227, Accuracy: 4838/7253 (67%)

-------------saving model--------------
Train Epoch: 1 [0/65435 (0%)]	Loss: 0.806610
Train Epoch: 1 [21800/65435 (33%)]	Loss: 1.025617
Train Epoch: 1 [43600/65435 (67%)]	Loss: 1.184834
Train Epoch: 1 [45780/65435 (100%)]	Loss: 0.859573

Valid set: Average loss: 0.9560, Accuracy: 5234/7253 (72%)

-------------saving model--------------
Train Epoch: 2 [0/65435 (0%)]	Loss: 0.724983
Train Epoch: 2 [21800/65435 (33%)]	Loss: 1.321963
Train Epoch: 2 [43600/65435 (67%)]	Loss: 0.765189
Train Epoch: 2 [45780/65435 (100%)]	Loss: 1.235174

Valid set: Average loss: 0.8747, Accuracy: 5377/7253 (74%)

-------------saving model--------------
Train Epoch: 3 [0/65435 (0%)]	Loss: 0.644023
Train Epoch: 3 [21800/65435 (33%)]	Loss: 0.676518
Train Epoch: 3

In [174]:
from chi_speech_command_dataset import SpeechCommandDataset

In [175]:
model = torch.load("1201_1.pth").to('cpu')

In [176]:
model.fc2 = nn.Linear(64,14).to('cpu')
print(model)
model = model.to(device)

QuatenarySincNet(
  (sincconv): _Layer(
    (conv): QuatSincConv1d(
      (w_quan): Quantize(num_of_bits=8)
    )
    (bn): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (htanh): Hardtanh(min_val=0.0, max_val=1.0)
    (quan): Quantize(num_of_bits=8)
    (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (distrloss): DistrLoss()
  )
  (features): ModuleList(
    (0): _Layer(
      (conv): QuaternaryConv1d(8, 16, kernel_size=(25,), stride=(2,), bias=False)
      (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (htanh): Hardtanh(min_val=0.0, max_val=1.0)
      (quan): Quantize(num_of_bits=8)
      (pool): MaxPool1d(kernel_size=10, stride=10, padding=0, dilation=1, ceil_mode=False)
      (distrloss): DistrLoss()
    )
    (1): _Layer(
      (conv): QuaternaryConv1d(16, 32, kernel_size=(9,), stride=(1,), bias=False)
      (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=T

In [177]:
chi_train_set = SpeechCommandDataset(data_path='/Desktop/CodeFolder/KWS/chinese/')
chi_val_set = SpeechCommandDataset(data_path='/Desktop/CodeFolder/KWS/chinese/', is_training=False)

print(chi_train_set.__len__())
print(chi_val_set.__len__())

417
126


In [178]:
chi_train_data_loader = DataLoader(chi_train_set, batch_size=5, shuffle=True, num_workers=4)
chi_val_data_loader = DataLoader(chi_val_set, batch_size=5, shuffle=True, num_workers=4)
learning_rate = 1e-2
decay_rate = learning_rate / ((epochs+10)/10)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
epochs = 40
acc = 0.0
for epoch in range(epochs):
#     model.train()
    train(model, chi_train_data_loader, optimizer, epoch)
    accuracy = valid(model, chi_val_data_loader)
    if accuracy > acc:
        acc = accuracy
        print("-------------saving model--------------")
        # save the model
        torch.save(model, "chi_1201_1.pth")

Train Epoch: 0 [0/417 (0%)]	Loss: 2.900572
Train Epoch: 0 [140/417 (33%)]	Loss: 1.978611
Train Epoch: 0 [280/417 (67%)]	Loss: 1.084961

Valid set: Average loss: 1.1349, Accuracy: 74/126 (59%)

-------------saving model--------------
Train Epoch: 1 [0/417 (0%)]	Loss: 1.101388
Train Epoch: 1 [140/417 (33%)]	Loss: 1.288153
Train Epoch: 1 [280/417 (67%)]	Loss: 1.064065

Valid set: Average loss: 1.1568, Accuracy: 78/126 (62%)

-------------saving model--------------
Train Epoch: 2 [0/417 (0%)]	Loss: 1.288226
Train Epoch: 2 [140/417 (33%)]	Loss: 0.638969
Train Epoch: 2 [280/417 (67%)]	Loss: 0.672189

Valid set: Average loss: 0.7406, Accuracy: 93/126 (74%)

-------------saving model--------------
Train Epoch: 3 [0/417 (0%)]	Loss: 0.537165
Train Epoch: 3 [140/417 (33%)]	Loss: 0.747289
Train Epoch: 3 [280/417 (67%)]	Loss: 0.574766

Valid set: Average loss: 0.8074, Accuracy: 88/126 (70%)

Train Epoch: 4 [0/417 (0%)]	Loss: 0.693800
Train Epoch: 4 [140/417 (33%)]	Loss: 0.408466
Train Epoch: 4 [280

In [179]:
print(model.fc2.weight.data)

tensor([[ 0.148437500000000000000000000000,  0.078125000000000000000000000000,
          0.125000000000000000000000000000, -0.132812500000000000000000000000,
         -0.507812500000000000000000000000, -0.218750000000000000000000000000,
          0.187500000000000000000000000000,  0.070312500000000000000000000000,
         -0.265625000000000000000000000000,  0.476562500000000000000000000000,
         -0.296875000000000000000000000000, -0.218750000000000000000000000000,
          0.023437500000000000000000000000,  0.187500000000000000000000000000,
         -0.570312500000000000000000000000, -0.367187500000000000000000000000,
         -0.242187500000000000000000000000, -0.054687500000000000000000000000,
          0.031250000000000000000000000000,  0.343750000000000000000000000000,
         -0.039062500000000000000000000000,  0.093750000000000000000000000000,
          0.351562500000000000000000000000,  0.593750000000000000000000000000,
         -0.492187500000000000000000000000, -0.30468

In [180]:
import torch.nn.utils.prune as prune
model = torch.load("chi_1201_1.pth").to('cpu')

In [181]:
parameters_to_prune = (
    (model.features[0].conv, 'weight'),
    (model.features[1].conv, 'weight'),
    (model.features[2].conv, 'weight'),
    (model.fc1, 'weight'),
    (model.fc2, 'weight'),
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)

In [182]:
print(
    "Sparsity in conv1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.features[0].conv.weight == 0))
        / float(model.features[0].conv.weight.nelement())
    )
)
print(
    "Sparsity in conv2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.features[1].conv.weight == 0))
        / float(model.features[1].conv.weight.nelement())
    )
)
print(
    "Sparsity in conv3.weight: {:.2f}%".format(
        100. * float(torch.sum(model.features[2].conv.weight == 0))
        / float(model.features[2].conv.weight.nelement())
    )
)
print(
    "Sparsity in full1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc1.weight == 0))
        / float(model.fc1.weight.nelement())
    )
)
print(
    "Sparsity in full2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc2.weight == 0))
        / float(model.fc2.weight.nelement())
    )
)

print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.features[0].conv.weight == 0)
            + torch.sum(model.features[1].conv.weight == 0)
            + torch.sum(model.features[2].conv.weight == 0)
            + torch.sum(model.fc1.weight == 0)
            + torch.sum(model.fc2.weight == 0)
        )
        / float(
            model.features[0].conv.weight.nelement()
            + model.features[1].conv.weight.nelement()
            + model.features[2].conv.weight.nelement()
            + model.fc1.weight.nelement()
            + model.fc2.weight.nelement()
        )
    )
)

Sparsity in conv1.weight: 24.50%
Sparsity in conv2.weight: 21.68%
Sparsity in conv3.weight: 19.84%
Sparsity in full1.weight: 16.67%
Sparsity in full2.weight: 44.20%
Global sparsity: 20.00%


In [183]:
%run /Desktop/Share/CUDA_DEVICE_setup.py -n 1

import sys
import torch
from A_speech_command_dataset import SpeechCommandDataset
from module import QuatSincConv1d
from module import SincConv1d
from module import Quantize, QuaternaryConv1d, QuaternaryLinear
from module import DistrLoss
import soundfile as sf
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch.nn.functional as F
print(torch.__version__)
from torchsummary import summary

using GPU ID:2,

1.6.0+cu92


In [184]:
from test_speech_command_dataset import SpeechCommandDataset

In [185]:
def test(model,data_loader,txt):
    label = {0:'1',1:'2',2:'3',3:'4',4:'5',5:'6',6:'7',7:'8',8:'9',9:'10',10:'up',11:'down',12:'open',13:'close'}
    with torch.no_grad():
        model.eval()
        valid_loss = 0
        correct = 0
        bs = test_data_loader.batch_size
        result = []
        for i, (data, target) in enumerate(test_data_loader):
            data, target = data.to('cpu'), target.to('cpu')
            output, loss1, loss2 = model(data)
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            arr = pred.data.cpu().numpy()
            for j in range(pred.size()[0]):
                file_name = txt[i*bs+j].split('_')[0]
                result.append((file_name,label[pred[j].cpu().numpy()[0]]))
    return result

In [186]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [187]:
chi_test_set = SpeechCommandDataset(data_path='/Desktop/CodeFolder/KWS/test/', is_training=False)
print(chi_test_set.__len__())

test_data_loader  = DataLoader(chi_test_set, batch_size=5, shuffle=False, num_workers=4) 

56


In [188]:
txt = [id.strip() for id in open('/Desktop/CodeFolder/KWS/test/valid_list.txt')]
test_model = torch.load("chi_1201_1.pth").to('cpu')
result = test(test_model, test_data_loader, txt)

In [189]:
print(result)

[('5', '5'), ('5', '5'), ('5', '5'), ('5', '5'), ('8', '2'), ('8', '8'), ('8', '8'), ('8', '8'), ('up', 'up'), ('up', 'up'), ('up', 'up'), ('up', 'up'), ('2', '2'), ('2', '2'), ('2', '2'), ('2', '2'), ('9', '9'), ('9', 'down'), ('9', '9'), ('9', '9'), ('1', '1'), ('1', '1'), ('1', '1'), ('1', '1'), ('6', '2'), ('6', '6'), ('6', '6'), ('6', '6'), ('down', 'down'), ('down', '10'), ('down', 'down'), ('down', 'down'), ('10', '10'), ('10', '10'), ('10', '10'), ('10', '10'), ('open', 'open'), ('open', 'open'), ('open', 'open'), ('open', '8'), ('4', '10'), ('4', '4'), ('4', '10'), ('4', '10'), ('3', '3'), ('3', 'up'), ('3', '3'), ('3', '3'), ('7', '7'), ('7', '10'), ('7', '7'), ('7', '7'), ('close', 'close'), ('close', 'close'), ('close', 'close'), ('close', 'close')]


In [190]:
err=0
for i in range(len(result)):
    if result[i][0] != result[i][1]:
        print("ans = ",result[i][0]," pred = ",result[i][1])
        err = err+1

ans =  8  pred =  2
ans =  9  pred =  down
ans =  6  pred =  2
ans =  down  pred =  10
ans =  open  pred =  8
ans =  4  pred =  10
ans =  4  pred =  10
ans =  4  pred =  10
ans =  3  pred =  up
ans =  7  pred =  10


In [191]:
print("accuracy = ", 1-err/len(result))

accuracy =  0.8214285714285714


In [192]:
#chi_1106.pth:  accuracy =  0.8928571428571429
#chi_1106_2.pth: accuracy =  0.9107142857142857

In [193]:
result_prune = test(model, test_data_loader, txt)
print(result_prune)

[('5', '5'), ('5', '5'), ('5', '5'), ('5', '5'), ('8', '8'), ('8', '8'), ('8', '8'), ('8', '8'), ('up', 'up'), ('up', 'up'), ('up', 'up'), ('up', 'up'), ('2', '2'), ('2', '2'), ('2', '2'), ('2', '2'), ('9', '9'), ('9', 'down'), ('9', '9'), ('9', '9'), ('1', '1'), ('1', '1'), ('1', '1'), ('1', '1'), ('6', '2'), ('6', '6'), ('6', '6'), ('6', '6'), ('down', 'down'), ('down', '10'), ('down', 'down'), ('down', 'down'), ('10', '4'), ('10', '10'), ('10', '4'), ('10', '10'), ('open', 'open'), ('open', 'open'), ('open', 'open'), ('open', '8'), ('4', '10'), ('4', '4'), ('4', '4'), ('4', '4'), ('3', '3'), ('3', 'up'), ('3', 'open'), ('3', '3'), ('7', '7'), ('7', '7'), ('7', '7'), ('7', '7'), ('close', 'close'), ('close', 'close'), ('close', 'close'), ('close', 'close')]


In [194]:
err=0
for i in range(len(result_prune)):
    if result_prune[i][0] != result_prune[i][1]:
        print("ans = ",result_prune[i][0]," pred = ",result_prune[i][1])
        err = err+1

print("accuracy = ", 1-err/len(result_prune))

ans =  9  pred =  down
ans =  6  pred =  2
ans =  down  pred =  10
ans =  10  pred =  4
ans =  10  pred =  4
ans =  open  pred =  8
ans =  4  pred =  10
ans =  3  pred =  up
ans =  3  pred =  open
accuracy =  0.8392857142857143


In [195]:
module = model.features[0].conv
prune.remove(module, 'weight')
print(list(module.named_parameters()))
module = model.features[1].conv
prune.remove(module, 'weight')
print(list(module.named_parameters()))
module = model.features[2].conv
prune.remove(module, 'weight')
print(list(module.named_parameters()))
module = model.fc1
prune.remove(module, 'weight')
print(list(module.named_parameters()))
module = model.fc2
prune.remove(module, 'weight')
print(list(module.named_parameters()))

[('weight', Parameter containing:
tensor([[[-0.140625000000000000000000000000, -0.546875000000000000000000000000,
          -0.000000000000000000000000000000,  ...,
           0.000000000000000000000000000000,  0.734375000000000000000000000000,
           0.000000000000000000000000000000],
         [-0.000000000000000000000000000000,  0.578125000000000000000000000000,
           0.281250000000000000000000000000,  ...,
          -0.000000000000000000000000000000,  0.515625000000000000000000000000,
          -0.000000000000000000000000000000],
         [ 0.250000000000000000000000000000, -0.171875000000000000000000000000,
           0.687500000000000000000000000000,  ...,
          -0.625000000000000000000000000000,  0.000000000000000000000000000000,
           0.546875000000000000000000000000],
         ...,
         [ 0.343750000000000000000000000000,  0.000000000000000000000000000000,
           0.359375000000000000000000000000,  ...,
          -0.000000000000000000000000000000, -0.25

In [196]:
torch.set_printoptions(precision=30)
for name, param in model.named_parameters():
    if param.requires_grad:
        print('name: \n',name)
        print('value: \n',param.data)

name: 
 sincconv.conv.low_hz_
value: 
 tensor([[  69.467529296875000000000000000000],
        [ 324.906158447265625000000000000000],
        [ 657.032836914062500000000000000000],
        [1153.684814453125000000000000000000],
        [1796.100097656250000000000000000000],
        [2680.237792968750000000000000000000],
        [3957.002197265625000000000000000000],
        [5621.648437500000000000000000000000]])
name: 
 sincconv.conv.band_hz_
value: 
 tensor([[ 299.732543945312500000000000000000],
        [ 375.838592529296875000000000000000],
        [ 481.105529785156250000000000000000],
        [ 681.962951660156250000000000000000],
        [ 902.480407714843750000000000000000],
        [1211.981201171875000000000000000000],
        [1671.821533203125000000000000000000],
        [2282.288330078125000000000000000000]])
name: 
 sincconv.conv.filters
value: 
 tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 

In [197]:
torch.save(model, "chi_1201_Q_pruned_20.pth")

In [198]:
model = torch.load("chi_1124_Q_pruned_20.pth").to('cpu')

In [204]:
import os, sys

In [205]:
torch.set_printoptions(threshold=sys.maxsize)
with open("/Desktop/CodeFolder/KWS/weights/weight.txt","w") as f:
    for name, param in model.named_parameters():
        if param.requires_grad:
            print('name: \n',name)
            f.write('\n*name: \n'+name)
            print('value: \n',param.data)
            f.write('\n*value: \n'+str(param.data))

name: 
 sincconv.conv.low_hz_
value: 
 tensor([[  30.000000000000000000000000000000],
        [ 293.618835449218750000000000000000],
        [ 652.436096191406250000000000000000],
        [1140.830078125000000000000000000000],
        [1805.593750000000000000000000000000],
        [2710.417968750000000000000000000000],
        [3941.993652343750000000000000000000],
        [5618.318359375000000000000000000000]], device='cuda:0')
name: 
 sincconv.conv.band_hz_
value: 
 tensor([[ 263.618835449218750000000000000000],
        [ 358.817291259765625000000000000000],
        [ 488.393981933593750000000000000000],
        [ 664.763671875000000000000000000000],
        [ 904.824218750000000000000000000000],
        [1231.575805664062500000000000000000],
        [1676.324584960937500000000000000000],
        [2281.681640625000000000000000000000]], device='cuda:0')
name: 
 sincconv.conv.filters
value: 
 tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [ ]:
model.summary()